In [71]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader

import os

base_dir = os.getcwd() 
file_path = os.path.join(base_dir, "..", "CorrectionDispersion", "dataset", "nps_simulated_dataset_gaussiano_2025-08-12.csv")
dataset = pd.read_csv(file_path)
tsf= pd.read_csv(os.path.join(base_dir, "..", "CorrectionDispersion", "dataset", "nps_simulated_dataset_tsfresh_2025-08-12.csv"))

print("Number of rows: ", dataset.shape[0])
print("Number of columns: ", dataset.shape[1])
print(dataset.info())
dataset.head(20)

Number of rows:  1000
Number of columns:  22
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   simulation_id       1000 non-null   int64  
 1   sensor_id           1000 non-null   int64  
 2   sensor_x            1000 non-null   float64
 3   sensor_y            1000 non-null   float64
 4   sensor_noise        1000 non-null   float64
 5   sensor_height       1000 non-null   float64
 6   sensor_is_fault     1000 non-null   bool   
 7   days                1000 non-null   int64  
 8   RH                  1000 non-null   float64
 9   humidify            1000 non-null   bool   
 10  days_simulation     1000 non-null   int64  
 11  wind_type           1000 non-null   object 
 12  wind_speed          1000 non-null   float64
 13  wind_dir            1000 non-null   object 
 14  stability_profile   1000 non-null   object 
 15  stability_v

,simulation_id,sensor_id,sensor_x,sensor_y,sensor_noise,sensor_height,sensor_is_fault,days,RH,humidify,...,wind_speed,wind_dir,stability_profile,stability_value,aerosol_type,source_x,source_y,source_h,emission_rate,real_concentration
0,0,0,121.0,18.0,0.0004,2.0,False,10,0.21,True,...,1.98,"297.50684357286747,191.49732006379136,53.45049...",CONSTANT,PasquillGiffordStability.VERY_STABLE,PIPERAZINE_ANALOGUES,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy
1,0,1,207.0,204.0,0.0005,2.0,False,10,0.21,True,...,1.98,"297.50684357286747,191.49732006379136,53.45049...",CONSTANT,PasquillGiffordStability.VERY_STABLE,PIPERAZINE_ANALOGUES,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy
2,0,2,105.0,0.0,0.0001,2.0,False,10,0.21,True,...,1.98,"297.50684357286747,191.49732006379136,53.45049...",CONSTANT,PasquillGiffordStability.VERY_STABLE,PIPERAZINE_ANALOGUES,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy
3,0,3,140.0,47.0,0.0002,2.0,False,10,0.21,True,...,1.98,"297.50684357286747,191.49732006379136,53.45049...",CONSTANT,PasquillGiffordStability.VERY_STABLE,PIPERAZINE_ANALOGUES,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy
4,0,4,371.0,481.0,0.0004,2.0,True,10,0.21,True,...,1.98,"297.50684357286747,191.49732006379136,53.45049...",CONSTANT,PasquillGiffordStability.VERY_STABLE,PIPERAZINE_ANALOGUES,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy
5,0,5,39.0,238.0,0.0003,2.0,False,10,0.21,True,...,1.98,"297.50684357286747,191.49732006379136,53.45049...",CONSTANT,PasquillGiffordStability.VERY_STABLE,PIPERAZINE_ANALOGUES,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy
6,0,6,243.0,353.0,0.0002,2.0,False,10,0.21,True,...,1.98,"297.50684357286747,191.49732006379136,53.45049...",CONSTANT,PasquillGiffordStability.VERY_STABLE,PIPERAZINE_ANALOGUES,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy
7,0,7,91.0,157.0,0.0002,2.0,False,10,0.21,True,...,1.98,"297.50684357286747,191.49732006379136,53.45049...",CONSTANT,PasquillGiffordStability.VERY_STABLE,PIPERAZINE_ANALOGUES,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy
8,0,8,268.0,148.0,0.0001,2.0,False,10,0.21,True,...,1.98,"297.50684357286747,191.49732006379136,53.45049...",CONSTANT,PasquillGiffordStability.VERY_STABLE,PIPERAZINE_ANALOGUES,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy
9,0,9,273.0,209.0,0.0002,2.0,False,10,0.21,True,...,1.98,"297.50684357286747,191.49732006379136,53.45049...",CONSTANT,PasquillGiffordStability.VERY_STABLE,PIPERAZINE_ANALOGUES,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy


In [72]:
print("Number of rows: ", tsf.shape[0])
print("Number of columns: ", tsf.shape[1])
print(tsf.info())
tsf.head(20)

Number of rows:  185280
Number of columns:  8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185280 entries, 0 to 185279
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   simulation_id  185280 non-null  int64  
 1   sensor_id      185280 non-null  int64  
 2   time           185280 non-null  float64
 3   conc           185280 non-null  float64
 4   wind_dir_x     185280 non-null  float64
 5   wind_dir_y     185280 non-null  float64
 6   wind_speed     185280 non-null  float64
 7   wind_type      185280 non-null  int64  
dtypes: float64(5), int64(3)
memory usage: 11.3 MB
None


,simulation_id,sensor_id,time,conc,wind_dir_x,wind_dir_y,wind_speed,wind_type
0,0,0,0.041667,2.863939e-10,0.461855,-0.886956,1.98,2
1,0,0,0.083333,0.000000e+00,-0.979934,-0.199322,1.98,2
2,0,0,0.125000,0.000000e+00,0.595517,0.803343,1.98,2
3,0,0,0.166667,0.000000e+00,-0.987262,-0.159104,1.98,2
4,0,0,0.208333,1.691768e-10,-0.879904,-0.475151,1.98,2
5,0,0,0.250000,0.000000e+00,0.832744,-0.553658,1.98,2
6,0,0,0.291667,3.198163e-10,-0.013049,-0.999915,1.98,2
7,0,0,0.333333,1.236331e-11,0.028243,-0.999601,1.98,2
8,0,0,0.375000,2.195897e-09,0.993757,-0.111565,1.98,2
9,0,0,0.416667,0.000000e+00,0.825323,0.564661,1.98,2


In [73]:
datset_copy= dataset.copy()

In [74]:
def convert_string_to_array(s):
    if isinstance(s, str):
        s = s.replace('[', '').replace(']', '')
        return np.fromstring(s, sep=',')
    return s

#datset_copy["contratio_series"] = datset_copy["contratio_series"].apply(convert_string_to_array) #type: ignore
datset_copy["wind_dir"] = datset_copy["wind_dir"].apply(convert_string_to_array) #type: ignore

In [75]:
datset_copy["wind_dir"]

0      [297.50684357286747, 191.49732006379136, 53.45...
1      [297.50684357286747, 191.49732006379136, 53.45...
2      [297.50684357286747, 191.49732006379136, 53.45...
3      [297.50684357286747, 191.49732006379136, 53.45...
4      [297.50684357286747, 191.49732006379136, 53.45...
                             ...                        
995    [19.24295130895386, 356.4341201335138, 357.108...
996    [19.24295130895386, 356.4341201335138, 357.108...
997    [19.24295130895386, 356.4341201335138, 357.108...
998    [19.24295130895386, 356.4341201335138, 357.108...
999    [19.24295130895386, 356.4341201335138, 357.108...
Name: wind_dir, Length: 1000, dtype: object

In [76]:
wind_type_map = {
    "CONSTANT": 1,
    "FLUCTUATING": 2,
    "PREVAILING": 3
}

pg_stability_map = {
    "PasquillGiffordStability.VERY_UNSTABLE": 1,
    "PasquillGiffordStability.MODERATELY_UNSTABLE": 2,
    "PasquillGiffordStability.SLIGHTLY_UNSTABLE": 3,
    "PasquillGiffordStability.NEUTRAL": 4,
    "PasquillGiffordStability.MODERATELY_STABLE": 5,
    "PasquillGiffordStability.VERY_STABLE": 6
}

nps_type_map = {
    "CANNABINOID_ANALOGUES": 0,
    "CATHINONE_ANALOGUES": 1,
    "PHENETHYLAMINE_ANALOGUES": 2,
    "PIPERAZINE_ANALOGUES": 3,
    "TRYPTAMINE_ANALOGUES": 4,
    "FENTANYL_ANALOGUES": 5,
    "OTHER_COMPOUNDS": 6
}


datset_copy['wind_type'] = datset_copy['wind_type'].astype(str)
datset_copy['stability_value'] = datset_copy['stability_value'].astype(str)
datset_copy['aerosol_type'] = datset_copy['aerosol_type'].astype(str)

datset_copy['wind_type'] = datset_copy['wind_type'].map(wind_type_map)
datset_copy['stability_value'] = datset_copy['stability_value'].map(pg_stability_map)
datset_copy['aerosol_type'] = datset_copy['aerosol_type'].map(nps_type_map)

datset_copy[["wind_type", "stability_value", "aerosol_type"]].head()

,wind_type,stability_value,aerosol_type
0,2,6,3
1,2,6,3
2,2,6,3
3,2,6,3
4,2,6,3


In [77]:
datset_copy.drop(columns=["humidify", "stability_profile", "sensor_height"], inplace=True)

In [78]:
datset_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   simulation_id       1000 non-null   int64  
 1   sensor_id           1000 non-null   int64  
 2   sensor_x            1000 non-null   float64
 3   sensor_y            1000 non-null   float64
 4   sensor_noise        1000 non-null   float64
 5   sensor_is_fault     1000 non-null   bool   
 6   days                1000 non-null   int64  
 7   RH                  1000 non-null   float64
 8   days_simulation     1000 non-null   int64  
 9   wind_type           1000 non-null   int64  
 10  wind_speed          1000 non-null   float64
 11  wind_dir            1000 non-null   object 
 12  stability_value     1000 non-null   int64  
 13  aerosol_type        1000 non-null   int64  
 14  source_x            1000 non-null   float64
 15  source_y            1000 non-null   float64
 16  source_

In [79]:
real_contration= datset_copy['real_concentration']
#datset_copy.drop(columns=["real_concentration"], inplace=True)

In [80]:
datset_copy.head()

,simulation_id,sensor_id,sensor_x,sensor_y,sensor_noise,sensor_is_fault,days,RH,days_simulation,wind_type,wind_speed,wind_dir,stability_value,aerosol_type,source_x,source_y,source_h,emission_rate,real_concentration
0,0,0,121.0,18.0,0.0004,False,10,0.21,10,2,1.98,"[297.50684357286747, 191.49732006379136, 53.45...",6,3,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy
1,0,1,207.0,204.0,0.0005,False,10,0.21,10,2,1.98,"[297.50684357286747, 191.49732006379136, 53.45...",6,3,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy
2,0,2,105.0,0.0,0.0001,False,10,0.21,10,2,1.98,"[297.50684357286747, 191.49732006379136, 53.45...",6,3,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy
3,0,3,140.0,47.0,0.0002,False,10,0.21,10,2,1.98,"[297.50684357286747, 191.49732006379136, 53.45...",6,3,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy
4,0,4,371.0,481.0,0.0004,True,10,0.21,10,2,1.98,"[297.50684357286747, 191.49732006379136, 53.45...",6,3,23.0,213.0,7.98,0.0044,sim_0_conc_real_2025-08-12.npy


In [81]:
datset_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   simulation_id       1000 non-null   int64  
 1   sensor_id           1000 non-null   int64  
 2   sensor_x            1000 non-null   float64
 3   sensor_y            1000 non-null   float64
 4   sensor_noise        1000 non-null   float64
 5   sensor_is_fault     1000 non-null   bool   
 6   days                1000 non-null   int64  
 7   RH                  1000 non-null   float64
 8   days_simulation     1000 non-null   int64  
 9   wind_type           1000 non-null   int64  
 10  wind_speed          1000 non-null   float64
 11  wind_dir            1000 non-null   object 
 12  stability_value     1000 non-null   int64  
 13  aerosol_type        1000 non-null   int64  
 14  source_x            1000 non-null   float64
 15  source_y            1000 non-null   float64
 16  source_

In [82]:
tsf["series_id"] = tsf["simulation_id"].astype(str) + "_" + tsf["sensor_id"].astype(str)
tsf.head(5)

,simulation_id,sensor_id,time,conc,wind_dir_x,wind_dir_y,wind_speed,wind_type,series_id
0,0,0,0.041667,2.863939e-10,0.461855,-0.886956,1.98,2,0_0
1,0,0,0.083333,0.000000e+00,-0.979934,-0.199322,1.98,2,0_0
2,0,0,0.125000,0.000000e+00,0.595517,0.803343,1.98,2,0_0
3,0,0,0.166667,0.000000e+00,-0.987262,-0.159104,1.98,2,0_0
4,0,0,0.208333,1.691768e-10,-0.879904,-0.475151,1.98,2,0_0


In [ ]:
tsf.drop(columns=["sensor_id", "simulation_id"], inplace=True)
tsf.head()

,time,conc,wind_dir_x,wind_dir_y,wind_speed,wind_type,series_id
0,0.041667,2.863939e-10,0.461855,-0.886956,1.98,2,0_0
1,0.083333,0.000000e+00,-0.979934,-0.199322,1.98,2,0_0
2,0.125000,0.000000e+00,0.595517,0.803343,1.98,2,0_0
3,0.166667,0.000000e+00,-0.987262,-0.159104,1.98,2,0_0
4,0.208333,1.691768e-10,-0.879904,-0.475151,1.98,2,0_0


In [104]:
from tsfresh import extract_features, select_features
from tsfresh.feature_extraction import EfficientFCParameters, ComprehensiveFCParameters

features_df = extract_features(tsf, column_id='series_id',  column_sort="time",
    default_fc_parameters=ComprehensiveFCParameters(),
    disable_progressbar=False)

features_df.head()

Feature Extraction: 100%|██████████| 30/30 [01:49<00:00,  3.66s/it]


,conc__variance_larger_than_standard_deviation,conc__has_duplicate_max,conc__has_duplicate_min,conc__has_duplicate,conc__sum_values,conc__abs_energy,conc__mean_abs_change,conc__mean_change,conc__mean_second_derivative_central,conc__median,...,wind_type__fourier_entropy__bins_5,wind_type__fourier_entropy__bins_10,wind_type__fourier_entropy__bins_100,wind_type__permutation_entropy__dimension_3__tau_1,wind_type__permutation_entropy__dimension_4__tau_1,wind_type__permutation_entropy__dimension_5__tau_1,wind_type__permutation_entropy__dimension_6__tau_1,wind_type__permutation_entropy__dimension_7__tau_1,wind_type__query_similarity_count__query_None__threshold_0.0,wind_type__mean_n_absolute_max__number_of_maxima_7
0_0,0.0,0.0,1.0,1.0,4.823822e-07,2.400741e-14,3.825763e-09,-1.198301e-12,6.016680e-13,4.666088e-11,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,2.0
0_1,0.0,0.0,1.0,1.0,4.672271e-06,8.177524e-12,2.781512e-08,-2.446573e-12,2.380704e-12,7.535098e-11,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,2.0
0_2,0.0,0.0,1.0,1.0,2.626195e-06,5.499703e-13,1.955085e-08,2.613471e-13,-4.146780e-13,4.179339e-12,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,2.0
0_3,0.0,0.0,1.0,1.0,9.989001e-08,6.599093e-16,7.493188e-10,-2.109400e-13,-6.343150e-13,5.023082e-11,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,2.0
0_5,0.0,0.0,1.0,1.0,7.937271e-06,2.455896e-11,6.631473e-08,-4.725806e-13,1.173536e-13,6.180564e-11,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,2.0


In [99]:
features_df.describe()

,wind_type__variance_larger_than_standard_deviation,wind_type__has_duplicate_max,wind_type__has_duplicate_min,wind_type__has_duplicate,wind_type__sum_values,wind_type__abs_energy,wind_type__mean_abs_change,wind_type__mean_change,wind_type__mean_second_derivative_central,wind_type__median,...,wind_speed__fourier_entropy__bins_5,wind_speed__fourier_entropy__bins_10,wind_speed__fourier_entropy__bins_100,wind_speed__permutation_entropy__dimension_3__tau_1,wind_speed__permutation_entropy__dimension_4__tau_1,wind_speed__permutation_entropy__dimension_5__tau_1,wind_speed__permutation_entropy__dimension_6__tau_1,wind_speed__permutation_entropy__dimension_7__tau_1,wind_speed__query_similarity_count__query_None__threshold_0.0,wind_speed__mean_n_absolute_max__number_of_maxima_7
count,772.0,772.0,772.0,772.0,772.000000,772.000000,772.0,772.0,772.0,772.000000,...,5.900000e+02,5.900000e+02,5.900000e+02,772.0,772.0,772.0,772.0,772.0,0.0,772.000000
mean,0.0,1.0,1.0,1.0,489.948187,1141.554404,0.0,0.0,0.0,2.041451,...,9.566083e-02,9.566083e-02,9.566083e-02,0.0,0.0,0.0,0.0,0.0,NaN,3.630155
std,0.0,0.0,0.0,0.0,184.303639,751.836605,0.0,0.0,0.0,0.767932,...,1.388956e-17,1.388956e-17,1.388956e-17,0.0,0.0,0.0,0.0,0.0,NaN,1.645310
min,0.0,1.0,1.0,1.0,240.000000,240.000000,0.0,0.0,0.0,1.000000,...,9.566083e-02,9.566083e-02,9.566083e-02,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,0.550000
25%,0.0,1.0,1.0,1.0,240.000000,240.000000,0.0,0.0,0.0,1.000000,...,9.566083e-02,9.566083e-02,9.566083e-02,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,2.450000
50%,0.0,1.0,1.0,1.0,480.000000,960.000000,0.0,0.0,0.0,2.000000,...,9.566083e-02,9.566083e-02,9.566083e-02,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,3.560000
75%,0.0,1.0,1.0,1.0,720.000000,2160.000000,0.0,0.0,0.0,3.000000,...,9.566083e-02,9.566083e-02,9.566083e-02,0.0,0.0,0.0,0.0,0.0,NaN,4.607500
max,0.0,1.0,1.0,1.0,720.000000,2160.000000,0.0,0.0,0.0,3.000000,...,9.566083e-02,9.566083e-02,9.566083e-02,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,7.990000


In [105]:
from sklearn.preprocessing import StandardScaler

# Rimuovi colonne costanti
features_df = features_df.loc[:, features_df.nunique() > 1]

# Sostituisci NaN
features_df = features_df.fillna(0)

# Rimuovi feature altamente correlate (>0.95)
corr = features_df.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
drop_cols = [col for col in upper.columns if any(upper[col] > 0.95)]
features_df.drop(columns=drop_cols, inplace=True)

print(f"[INFO] Feature dopo pulizia: {features_df.shape}")

dataset["series_id"] = dataset["simulation_id"].astype(str) + "_" + dataset["sensor_id"].astype(str)

# mappa target (source_x per ogni series_id)
target_map = dataset.set_index("series_id")["source_x"]

# Filtriamo target solo per le serie presenti nelle feature
target_for_selection = target_map.loc[features_df.index]

# Selezione feature rilevanti
X_filtered = select_features(features_df, target_for_selection, fdr_level=0.05)

print(f"[INFO] Feature dopo selezione: {X_filtered.shape}")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filtered)

print(f"[INFO] Dati finali scalati: {X_scaled.shape}")

[INFO] Feature dopo pulizia: (772, 1698)
[INFO] Feature dopo selezione: (772, 244)
[INFO] Dati finali scalati: (772, 244)


In [111]:
X_scaled = pd.DataFrame(X_scaled, index=X_filtered.index, columns=X_filtered.columns)
X_scaled.columns

Index(['wind_dir_x__fft_coefficient__attr_"real"__coeff_53',
       'wind_dir_x__fft_coefficient__attr_"imag"__coeff_27',
       'wind_dir_x__fft_coefficient__attr_"imag"__coeff_93',
       'wind_dir_x__fft_coefficient__attr_"imag"__coeff_73',
       'wind_dir_y__fft_coefficient__attr_"imag"__coeff_46',
       'wind_dir_x__fft_coefficient__attr_"angle"__coeff_93',
       'wind_dir_y__change_quantiles__f_agg_"mean"__isabs_False__qh_0.8__ql_0.4',
       'wind_dir_x__energy_ratio_by_chunks__num_segments_10__segment_focus_5',
       'wind_dir_y__fft_coefficient__attr_"angle"__coeff_74',
       'wind_dir_y__fft_coefficient__attr_"angle"__coeff_46',
       ...
       'conc__change_quantiles__f_agg_"var"__isabs_True__qh_0.2__ql_0.0',
       'conc__change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0',
       'wind_speed__large_standard_deviation__r_0.05',
       'wind_dir_y__fft_coefficient__attr_"abs"__coeff_90',
       'wind_dir_y__fft_coefficient__attr_"abs"__coeff_62',
       'wind_

In [112]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

# Funzione RMSE medio sulle due coordinate
def rmse_multioutput(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse_multioutput, greater_is_better=False)

# Target: X e Y della sorgente
Y = dataset.set_index("series_id").loc[X_filtered.index, ["source_x", "source_y"]]


In [113]:
Y

,source_x,source_y
0_0,23.0,213.0
0_1,23.0,213.0
0_2,23.0,213.0
0_3,23.0,213.0
0_5,23.0,213.0
...,...,...
9_5,281.0,107.0
9_6,281.0,107.0
9_7,281.0,107.0
9_8,281.0,107.0


In [118]:
# Modello multivariato
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=200, random_state=42))

# K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

rmse_scores = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X_filtered), start=1):
    X_train, X_test = X_filtered.iloc[train_idx], X_filtered.iloc[test_idx]
    y_train, y_test = Y.iloc[train_idx], Y.iloc[test_idx]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # RMSE fold
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse_scores.append(rmse)
    
    print(f"\n[Fold {fold}] RMSE: {rmse:.3f}")
    print("Predizioni (x, y) vs. Reale:")
    for pred, real in zip(y_pred[:5], y_test.values[:5]):  # Mostra solo le prime 5
        print(f"Pred: [{pred[0]:.3f}, {pred[1]:.3f}] | Reale: [{real[0]}, {real[1]}]")

# Risultato finale
print(f"\nRMSE medio CV: {np.mean(rmse_scores):.3f} ± {np.std(rmse_scores):.3f}")



[Fold 1] RMSE: 43.090
Predizioni (x, y) vs. Reale:
Pred: [23.000, 213.735] | Reale: [23.0, 213.0]
Pred: [23.000, 213.295] | Reale: [23.0, 213.0]
Pred: [409.000, 177.250] | Reale: [409.0, 176.0]
Pred: [258.240, 259.580] | Reale: [262.0, 260.0]
Pred: [385.070, 417.420] | Reale: [386.0, 421.0]

[Fold 2] RMSE: 44.052
Predizioni (x, y) vs. Reale:
Pred: [23.000, 212.535] | Reale: [23.0, 213.0]
Pred: [408.205, 177.445] | Reale: [409.0, 176.0]
Pred: [407.685, 178.030] | Reale: [409.0, 176.0]
Pred: [259.225, 259.615] | Reale: [262.0, 260.0]
Pred: [271.075, 258.725] | Reale: [262.0, 260.0]

[Fold 3] RMSE: 34.825
Predizioni (x, y) vs. Reale:
Pred: [23.000, 213.305] | Reale: [23.0, 213.0]
Pred: [406.660, 177.555] | Reale: [409.0, 176.0]
Pred: [409.740, 175.240] | Reale: [409.0, 176.0]
Pred: [251.745, 258.745] | Reale: [262.0, 260.0]
Pred: [260.725, 258.745] | Reale: [262.0, 260.0]

[Fold 4] RMSE: 50.509
Predizioni (x, y) vs. Reale:
Pred: [23.000, 215.940] | Reale: [23.0, 213.0]
Pred: [23.000, 216

In [117]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Convertiamo i dati in tensori
X_tensor = torch.tensor(X_scaled.values, dtype=torch.float32)
Y_tensor = torch.tensor(Y.values, dtype=torch.float32)

dataset_torch = TensorDataset(X_tensor, Y_tensor)

# MLP
class MLP(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)  # Output: source_x, source_y
        )

    def forward(self, x):
        return self.layers(x)

# Training
model = MLP(X_scaled.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

loader = DataLoader(dataset_torch, batch_size=32, shuffle=True)

for epoch in range(100):
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        pred = model(X_batch)
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1} - Loss: {loss.item():.4f}")


Epoch 10 - Loss: 13008.6348
Epoch 20 - Loss: 333.2252
Epoch 30 - Loss: 1379.5560
Epoch 40 - Loss: 1235.4573
Epoch 50 - Loss: 11689.4355
Epoch 60 - Loss: 3611.4294
Epoch 70 - Loss: 3203.3838
Epoch 80 - Loss: 1857.6367
Epoch 90 - Loss: 677.9065
Epoch 100 - Loss: 2270.6223
